In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm

In [31]:
df = pd.DataFrame({'Well': ['1A', '2A', '1B', '2B', '1C', '2C', '1D', '2D', '1E', '2E', '1F', '2F', '1G', '2G', '1H', '2H'],
                   'Gas Production': [4494, 9182, 2989, 12258, 8365, 8742, 7447, 3840, 3290, 7939, 14737, 2874, 11155, 6076, 7081, 6468],
                   'Unused Production': [4494, 9182, 2989, 12258, 8365, 8742, 7447, 3840, 3290, 7939, 14737, 2874, 11155, 6076, 7081, 6468],
                   'Distance': [62.0, 85.8, 21.0, 84.7, 17.0, 2.4, 73.4, 42.0, 6.2, 98.8, 91.0, 13.0, 59.0, 87.0, 89.3, 36.0],
                   'Deployed Module': [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]})

trucks = pd.DataFrame({'Truck ID': [1],
                       'Available': [False],
                       'Active Contract': [False],
                       'Years until expiration': [0]})

In [3]:
df

,Well,Gas Production,Unused Production,Distance,Deployed Module
0,1A,4494,4494,62.0,None
1,2A,9182,9182,85.8,None
2,1B,2989,2989,21.0,None
3,2B,12258,12258,84.7,None
4,1C,8365,8365,17.0,None
5,2C,8742,8742,2.4,None
6,1D,7447,7447,73.4,None
7,2D,3840,3840,42.0,None
8,1E,3290,3290,6.2,None
9,2E,7939,7939,98.8,None


In [44]:
def law_of_cosines(len1, len2, angle):
    return np.sqrt(len1**2 + len2**2 -2*len1*len2*np.cos(np.radians(angle)))

def distance(x, i, j):
    return law_of_cosines(x['Distance'][i], x['Distance'][j], np.abs(((i-j)*22.5)))

def distance_iter(x):
    all = []
    well_1 = []
    well_2 = []
    for i in range(len(x)):
        for j in range(len(x)):
            if i != j:
                k = distance(x, i, j)
                all.append(k)
                well_1.append(i)
                well_2.append(j)
    return np.array(all), np.array(well_1), np.array(well_2) 

def prod_decline(x, year):
    if year < 20 and year >= 0:
        production = [x]
        y = len(range(year,19))
        while y > 0:
            if 19 - y >= year + 1:
                x = x * 0.65
                production.append(x)
            else:
                production.append(x)
            y-=1
        return production
    else:
        print('Year value must be between 0 and 20')

def unit_size(size, density, lpg, naptha, diesel, w=0):
    base = density * (1.25 * lpg + 1.79 * naptha + 2.14 * diesel) * 365 * 0.8
    if w == 0:
        if size == 'small':
            return 500 * base #$/gal*yr & MSCF to gallons
        elif size == 'medium':
            return 2500 * base
        elif size == 'large':
            return 5000 * base
    else:
        if size == 'small':
            return w * base #$/gal*yr & MSCF to gallons
        elif size == 'medium':
            return w * base
        elif size == 'large':
            return w * base
        
def truck_contracts(load, distance):
    x = int(trucks['Truck ID'] - 1)
    y = int(load / 5000) + (load % 5000 > 0)
    updated_roster = trucks
    for i in range(y):
        # Claim truck and take off roster
        if trucks['Available'][x] == True and trucks['Active Contract'][x] == True:
            trucks['Available'][x] = False

        # Renew contract if expired
        elif trucks['Active Contract'][x] == False or trucks['Years until expiration'][x] == 0:
            trucks['Active Contract'][x] = True
            trucks['Years until expiration'][x] = 5
            print('Contract for truck #{} has been renewed.'.format(truck_id))

        # Add new truck contract if no trucks available
        elif trucks['Available'].all() == False:
            updated_roster = updated_roster.append({'Truck ID': len(updated_roster)+1,
                            'Available': False,
                            'Active Contract': True,
                            'Years until expiration': 5}, ignore_index=True)             

    return distance * 1.25 * y, updated_roster


# Will need to fix, module argument
def redeploy_module(module, distance): # move a module from one well to another well
    if module == 's':
        return -(500 * 3 * distance + (small(1, 0.33, 0.33, 0.33)) * 30/292)
    elif module == 'm':
        return -(2500 * 3 * distance + (medium(1, 0.33, 0.33, 0.33)) * 30/292)
    elif module == 'l':
        return -(5000 * 3 * distance + (large(1, 0.33, 0.33, 0.33)) * 30/292)

def deploy_unit(module_size, miles, year, well): # deploy a module to the first well
    module_capacity = 0
    if module_size == 'small':
        module_capacity = 500
        well_capacity = prod_decline(well, year)
        result = []
        if well_capacity[0] < module_capacity:
            result.append(unit_size(module_size, 1, 0.33, 0.33, 0.33, w=well_capacity[0])*365*0.8 - miles*500*3)
        else:
            result.append(unit_size(module_size, 1, 0.33, 0.33, 0.33)*365*0.8 - miles*500*3)
        for w in well_capacity[1:]:
            if w > module_capacity:
                result.append(unit_size(module_size, 1, 0.33, 0.33, 0.33)*365*0.8)
            else:
                result.append(unit_size(module_size, 1, 0.33, 0.33, 0.33,w=w)*365*0.8)
        return result
    elif module_size == 'medium':
        module_capacity = 2500
        well_capacity = prod_decline(well, year)
        result = []
        if well_capacity[0] < module_capacity:
            result.append(unit_size(module_size, 1, 0.33, 0.33, 0.33, w=well_capacity[0])*365*0.8 - miles*500*3)
        else:
            result.append(unit_size(module_size, 1, 0.33, 0.33, 0.33)*365*0.8 - miles*500*3)
        for w in well_capacity[1:]:
            if w > module_capacity:
                result.append(unit_size(module_size, 1, 0.33, 0.33, 0.33)*365*0.8)
            else:
                result.append(unit_size(module_size, 1, 0.33, 0.33, 0.33,w=w)*365*0.8)
        return result
    elif module_size == 'large':
        module_capacity = 5000
        well_capacity = prod_decline(well, year)
        result = []
        if well_capacity[0] < module_capacity:
            result.append(unit_size(module_size, 1, 0.33, 0.33, 0.33, w=well_capacity[0])*365*0.8 - miles*500*3)
        else:
            result.append(unit_size(module_size, 1, 0.33, 0.33, 0.33)*365*0.8 - miles*500*3)
        for w in well_capacity[1:]:
            if w > module_capacity:
                result.append(unit_size(module_size, 1, 0.33, 0.33, 0.33)*365*0.8)
            else:
                result.append(unit_size(module_size, 1, 0.33, 0.33, 0.33,w=w)*365*0.8)
        return result

def parallel_units(number_of_small, number_of_medium, number_of_large):
    modules = pd.DataFrame({'Module ID': [],
                            'Module size': [],
                        'Available': bool([])})

    equipment_cost = number_of_small*500_000 + number_of_medium*2_500_000 + number_of_large*5_000_000 # Need to update equipment pricing
    i = 0
    while i < len(range(number_of_large)):
        modules = modules.append({'Module ID': 'large {}'.format(i+1),
                                  'Module size': 'large',
                                  'Available': True,
                                  'Capacity': 5000}, ignore_index=True)
        i+=1
    i = 0
    while i < len(range(number_of_medium)):
        modules = modules.append({'Module ID': 'medium {}'.format(i+1),
                                  'Module size': 'medium',
                                  'Available': True,
                                  'Capacity': 2500}, ignore_index=True)
        i+=1
    i = 0
    while i < len(range(number_of_small)):
        modules = modules.append({'Module ID': 'small {}'.format(i+1),
                                  'Module size': 'small',
                                  'Available': True,
                                  'Capacity': 500}, ignore_index=True)
        i+=1
    return modules, equipment_cost
    
def check_redeploy(year):
    all_wells, well_a, well_b = distance_iter(df)
    i = 0
    report = []
    while units_in_circulation['Available'].any() == True:
        for unit in units_in_circulation['Module size']:
            for a in well_a:
                for b in well_b:
                    if units_in_circulation['Available'][i] == True and units_in_circulation['Module size'][i] == large: # deploy large unit if it is available
                        yearly_report = deploy_unit(large, distance(df, a, b),year-20,df['Gas Production'][b])
                        j = len(yearly_report)
                        x = yearly_report[j-1]
                        report.append(x)
                    elif units_in_circulation['Available'][i] == True and units_in_circulation['Module size'][i] == medium: # deploy medium unit if it is available
                        yearly_report= deploy_unit(medium, distance(df, a, b),year,df['Gas Production'][b])
                        j = len(yearly_report)  
                        report.append(yearly_report[j - 1])
                    elif units_in_circulation['Available'][i] == True and units_in_circulation['Module size'][i] == small: # deploy small unit if it is available
                        yearly_report= deploy_unit(small, distance(df, a, b),year,df['Gas Production'][b])
                        j = len(yearly_report)
                        report.append(yearly_report[j - 1])
            units_in_circulation['Available'][i] = False
            i+=1
    return report


In [35]:
# Deploy units to initial location (Ax optimize later)
year = 19

for i in range(len(units_in_circulation)):
    for j in range(len(df)):      
        if units_in_circulation['Available'].any() == True:
            for unit in units_in_circulation['Module ID']:
                if units_in_circulation['Available'][i] == True and df['Unused Production'][j] >= units_in_circulation['Capacity'][i]:
                    print('{} deployed at well {}'.format(units_in_circulation['Module ID'][i], df['Well'][j]))
                    revenue = deploy_unit(units_in_circulation['Module size'][i], df['Distance'][j], year, df['Gas Production'][j])
                    units_in_circulation['Available'][i] = False
                    df['Deployed Module'][i] = units_in_circulation['Module size'][i]

                    if df['Unused Production'][j] >= units_in_circulation['Capacity'][i]:
                        df['Unused Production'][j] -= units_in_circulation['Capacity'][i]
                    else:
                        df['Unused Production'][j] = 0
            

medium 1 deployed at well 1A


<ipython-input-35-93348216d482>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  units_in_circulation['Available'][i] = False
<ipython-input-35-93348216d482>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Deployed Module'][i] = units_in_circulation['Module size'][i]
<ipython-input-35-93348216d482>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unused Production'][j] -= units_in_circulation['Capacity'][i]


In [67]:
year = 0
while year < 20:
    for i in range(len(units_in_circulation)):
        for j in range(len(df)):      
            if units_in_circulation['Available'].any() == True:
                for unit in units_in_circulation['Module ID']:
                    if units_in_circulation['Available'][i] == True and df['Unused Production'][j] >= units_in_circulation['Capacity'][i]:
                        print('{} deployed at well {}'.format(units_in_circulation['Module ID'][i], df['Well'][j]))
                        revenue = deploy_unit(units_in_circulation['Module size'][i], df['Distance'][j], year, df['Gas Production'][j])
                        units_in_circulation['Available'][i] = False
                        df['Deployed Module'][i] = units_in_circulation['Module size'][i]

                        if df['Unused Production'][j] >= units_in_circulation['Capacity'][i]:
                            df['Unused Production'][j] -= units_in_circulation['Capacity'][i]
                        else:
                            df['Unused Production'][j] = 0

    if df['Deployed Module'][j] != None:
        decline = prod_decline(df['Gas Production'][j], year)

    df['Gas Production'][j] = decline[year]
    print(df['Gas Production'][j])
            #df['Gas Production'][i] = decline[year]
    year += 1
df

0
0
0
0
0
0
0
0
0
0


<ipython-input-67-e63075d65ea2>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gas Production'][j] = decline[year]


IndexError: list index out of range

In [69]:
print(year,j, decline)

10 15 [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [14]:
units_in_circulation, df

(  Module ID Module size  Available  Capacity
 0  medium 1      medium       True    2500.0,
    Well  Gas Production  Unused Production  Distance Deployed Module
 0    1A            4494               4494      62.0            None
 1    2A            9182               9182      85.8            None
 2    1B            2989               2989      21.0            None
 3    2B           12258              12258      84.7            None
 4    1C            8365               8365      17.0            None
 5    2C            8742               8742       2.4            None
 6    1D            7447               7447      73.4            None
 7    2D            3840               3840      42.0            None
 8    1E            3290               3290       6.2            None
 9    2E            7939               7939      98.8            None
 10   1F           14737              14737      91.0            None
 11   2F            2874               2874      13.0            No

In [32]:
year = 20
dist, well_a, well_b = distance_iter(df)
annual = []

number_of_small = 0
number_of_medium = 1
number_of_large = 0

units_in_circulation, FCI = parallel_units(number_of_small, number_of_medium, number_of_large)
#deploy_unit()
#update_unit()
#redeploy_module()



In [ ]:
'''
all_wells, well_a, well_b = distance_iter(df)
year = 1
i = 0
report = []
for a in tqdm.tqdm(well_a):
                for b in well_b:
                    if units_in_circulation['Available'][i] == True and units_in_circulation['Module size'][i] == 'large': # deploy large unit if it is available
                        yearly_report = deploy_unit('large', distance(df, a, b),year,df['Gas Production'][b])
                        j = len(yearly_report)
                        x = yearly_report[j-1]
                        report.append(x)
                        units_in_circulation['Available'][i] = False
                    
                    elif units_in_circulation['Available'][i] == True and units_in_circulation['Module size'][i] == medium: # deploy medium unit if it is available
                        yearly_report= deploy_unit(medium, distance(df, a, b),year,df['Gas Production'][b])
                        j = len(yearly_report)  
                        report.append(yearly_report[j - 1])
                        units_in_circulation['Available'][i] == False
                    elif units_in_circulation['Available'][i] == True and units_in_circulation['Module size'][i] == small: # deploy small unit if it is available
                        yearly_report= deploy_unit(small, distance(df, a, b),year,df['Gas Production'][b])
                        j = len(yearly_report)
                        report.append(yearly_report[j - 1])
                        units_in_circulation['Available'][i] == False
                    #print(yearly_report, report, a, b)
                    
print(len(yearly_report),a, b)
'''


  0%|          | 0/240 [00:00<?, ?it/s]<ipython-input-710-d35d1c7f5218>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  units_in_circulation['Available'][i] = False
100%|██████████| 240/240 [00:00<00:00, 264.27it/s]

20 15 14
